# Contents
1. Introduction <br>
    1.1. Import Libraries<br>
    1.2. Terminology <br>
    
    
2. Gathering and Cleaning Data <br> 
    2.1. Player Data <br>
    2.2. Fixture Data <br>
    2.3. Game Data <br>
    
    
3. Data Engineering (Team Data) <br>
    3.1. Team npxG <br>
    3.2. Team npxG (Home Fixtures) <br>
    3.3. Team npxG (Away Fixtures) <br>
    3.4. Team npxGA <br>
    3.5. Team npxGA (Home Fixtures) <br>
    3.6. Team npxGA (Away Fixtures) <br>
    
    
4. Home/Away Modifiers for npxG/npxGA <br>
    4.1. Home/Away Modifier Calculation <br>
    4.2. Dictionaries <br> 


5. Team Fixture Tables <br>
    5.1. Home Fixture Table <br>
    5.2. Away Fixture Table <br>
    5.3. Home Fixture Table - Insert Stats <br> 
    5.4. Away Fixture Table - Insert Stats <br>
    5.5. Finalizing Fixture Table <br>
    
    
6. Data Engineering (Player Data) - Fuzzy Matching Included <br>
    6.1. Player npxG and xAG <br> 
    6.2. Fuzzy Matching <br>
    
    
7. Players Fixture Tables <br>
    7.1. Merge with Team Data <br>
    7.2. Home Fixture Table <br>
    7.3. Away Fixture Table <br>
    7.4. Finalizing Fixture Table <br>
    
    
8. Final Dataframes <br>
    8.1. Team Stats <br>
    8.2. Team Fixture Tables <br>
    8.3. Team npxG Predictions <br> 
    8.4. Player npxG and xAG Predictions <br>

# 1. Introduction 

Welcome to our Fantasy Premier League (FPL) kernel! We will be gathering data from the official FPL API and popular football stats website FBref to produce a prediction system for expected goals, assists and goals conceeded for each team and player in the premier league for the upcoming five gameweeks.   

## 1.1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import soccerdata as sd
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from itertools import repeat
from fpl import FPL
import requests, json
from thefuzz import fuzz, process
from sklearn.preprocessing import MinMaxScaler

[02/10/23 15:08:38] INFO     No custom team name replacements found. You can configure these in       ]8;id=695210;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=972391;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_config.py#85\85]8;;\
                             C:\Users\waqas\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=778498;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=364067;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_config.py#159\159]8;;\
                             C:\Users\waqas\soccerdata\config\league_dict.json.                                    

## 1.2. Terminology


### Non-Penalty Expected Goals (npxG)
Non-penalty expected goals (npxG) is the probability that a non-penalty shot will result in a goal. This metric is known to be the best predictor for future goals. 

### Non-Penalty Expected Goals Against (npxGA)
Non-penalty expected goals against (npxGA) is the probability that an opponent's non-penalty shot will result in a goal conceeded.

### Expected Assist (xAG)
Expected assists (xAG) is the probability that a given pass will become an assist to a goal.

# 2. Gathering and Cleaning Data

We need to gather data from the FPL API and the FBref site. The former will be accessed directly. The latter will be accessed using a python library (soccerdata) which serves as a wrapper. 

## 2.1. Player Data


We'll be gathering 3 datasets from the official FPL API and joining them;

Team

Player

Player position 

In [2]:
#Calling FPL API for various player-related datasets.
base_url = 'https://fantasy.premierleague.com/api/'
r = requests.get(base_url+'bootstrap-static/').json()

teams = pd.json_normalize(r['teams'])
players = pd.json_normalize(r['elements'])
players_pos = pd.json_normalize(r['element_types'])

teams.head(3)
players.head(3)
players_pos.head(3)

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1090,1070,1130,1050,1080,127


,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,NaN,84450,-1,1,-1,1,1,3,6.6,...,239,138,79,27,59,26,101,34,1.06,0.42
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,...,625,320,491,183,387,169,208,65,0.81,0.00
2,NaN,NaN,156074,0,0,-3,3,0,2,0.6,...,598,169,339,119,421,157,433,163,0.00,0.00


,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],77
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],250
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],321


Alot of bulk, let's clean it up.

Team data filtered for: ID and corresponding Team Name.

In [3]:
dfTeam=teams[['id', 'name']]
dfTeam.head(3)

,id,name
0,1,Arsenal
1,2,Aston Villa
2,3,Bournemouth


Player position data filtered for: ID and corresponding position name. 

In [4]:
dfPlayers_pos=players_pos[['id','plural_name_short']]
dfPlayers_pos=dfPlayers_pos.rename(columns={'plural_name_short':'Position'})
dfPlayers_pos.head(3)

,id,Position
0,1,GKP
1,2,DEF
2,3,MID


Player data filtered for: Team (which is in ID form), Position (element_type), First Name and Last Name (combined to 'Player').

In [5]:
dfPlayers=players[['team', 'element_type', 'first_name', 'second_name']]
dfPlayers['Player'] = dfPlayers['first_name'] + ' ' + dfPlayers['second_name']
dfPlayers=dfPlayers.rename(columns={'team':'id'}) 
dfPlayers.head(3)

,id,element_type,first_name,second_name,Player
0,1,3,Granit,Xhaka,Granit Xhaka
1,1,3,Mohamed,Elneny,Mohamed Elneny
2,1,2,Rob,Holding,Rob Holding


In the dfPlayers dataframe, we need to replace ID with the respective team ID and similarly replace element type with the respective position.

In [6]:
#Make dictionaries for Team ID+Names & Player Position ID+Names.
team_id_dict=pd.Series(dfTeam.name.values,index=dfTeam.id).to_dict()
player_pos_id_dict=pd.Series(dfPlayers_pos.Position.values,index=dfPlayers_pos.id).to_dict()

Apply dictionaries to make new columns giving actual team names and positions.

In [7]:
dfPlayers["Team"] = dfPlayers["id"].apply(lambda x: team_id_dict.get(x))
dfPlayers["Position"] = dfPlayers["element_type"].apply(lambda x: player_pos_id_dict.get(x))
dfPlayers_FPL=dfPlayers.drop(columns=['element_type', 'id', 'first_name', 'second_name'])

dfPlayers_FPL.head()

,Player,Team,Position
0,Granit Xhaka,Arsenal,MID
1,Mohamed Elneny,Arsenal,MID
2,Rob Holding,Arsenal,DEF
3,Thomas Partey,Arsenal,MID
4,Martin Ødegaard,Arsenal,MID


Our player data is cleaned up! 

## 2.2. Fixtures Data

Our fixture data will be accessed from the official API again. Like before, we're going to see alot of excess that needs to be cleaned up. 

In [8]:
#Calling FPL API for fixture data. 
base_url = 'https://fantasy.premierleague.com/api/'
r = requests.get(base_url+'fixtures?future=1').json()
fixtures = pd.json_normalize(r)
fixtures.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2293039,23,False,False,230,2023-02-11T12:30:00Z,0,False,False,6,None,19,None,[],3,3,75140
1,2293031,23,False,False,221,2023-02-11T15:00:00Z,0,False,False,4,None,1,None,[],2,4,75131
2,2293032,23,False,False,223,2023-02-11T15:00:00Z,0,False,False,5,None,7,None,[],3,3,75133
3,2293033,23,False,False,224,2023-02-11T15:00:00Z,0,False,False,16,None,9,None,[],2,2,75134
4,2293035,23,False,False,226,2023-02-11T15:00:00Z,0,False,False,18,None,10,None,[],3,3,75136


Columns team_h and team_a give the respective Home and Away team in ID form. I will use the dictionary made in the previous section to replace ID's with their respective Team Names. 

In [9]:
fixtures['Home Team']=fixtures["team_h"].apply(lambda x: team_id_dict.get(x))
fixtures['Away Team']=fixtures["team_a"].apply(lambda x: team_id_dict.get(x))
fixtures.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,Home Team,Away Team
0,2293039,23,False,False,230,2023-02-11T12:30:00Z,0,False,False,6,None,19,None,[],3,3,75140,West Ham,Chelsea
1,2293031,23,False,False,221,2023-02-11T15:00:00Z,0,False,False,4,None,1,None,[],2,4,75131,Arsenal,Brentford
2,2293032,23,False,False,223,2023-02-11T15:00:00Z,0,False,False,5,None,7,None,[],3,3,75133,Crystal Palace,Brighton
3,2293033,23,False,False,224,2023-02-11T15:00:00Z,0,False,False,16,None,9,None,[],2,2,75134,Fulham,Nott'm Forest
4,2293035,23,False,False,226,2023-02-11T15:00:00Z,0,False,False,18,None,10,None,[],3,3,75136,Leicester,Spurs


Remove excess columns and replace team names (will be important later on). 

In [10]:
fixtures=fixtures[['event', 'Home Team', 'Away Team']]
fixtures=fixtures.rename(columns={'event':'GameWeek'})
fixtures['Home Team'] = fixtures['Home Team'].replace({'Newcastle':'Newcastle Utd', 'Spurs':'Tottenham','Leeds':'Leeds United', 'Leicester':'Leicester City', 'Man City':'Manchester City', 'Man Utd':'Manchester Utd', "Nott'm Forest":"Nott'ham Forest"})
fixtures['Away Team'] = fixtures['Away Team'].replace({'Newcastle':'Newcastle Utd', 'Spurs':'Tottenham','Leeds':'Leeds United', 'Leicester':'Leicester City', 'Man City':'Manchester City', 'Man Utd':'Manchester Utd', "Nott'm Forest":"Nott'ham Forest"})
dfFixtures=fixtures
dfFixtures.head()

,GameWeek,Home Team,Away Team
0,23,West Ham,Chelsea
1,23,Arsenal,Brentford
2,23,Crystal Palace,Brighton
3,23,Fulham,Nott'ham Forest
4,23,Leicester City,Tottenham


Our fixture data is cleaned up! 

## 2.3. Game Data 

Finally we need to bring in our games database. For this, we'll be using the aforementioned scraper (soccerdata) to bring in all games from the previous and current season from FBref. 

Be warned, this can take about 5 minutes.

In [2]:
#Create scraper class objects from 'soccerdata' library for scraping seasons 2021/2022 and 2022/2023.
fbref = sd.FBref(leagues='ENG-Premier League', seasons='2021')
fbref1 = sd.FBref(leagues='ENG-Premier League', seasons='2022')

[02/10/23 15:08:45] INFO     Saving cached data to C:\Users\waqas\soccerdata\data\FBref               ]8;id=799846;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=631868;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_common.py#89\89]8;;\

                    INFO     Saving cached data to C:\Users\waqas\soccerdata\data\FBref               ]8;id=299860;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=335527;file://C:\Users\waqas\anaconda3\lib\site-packages\soccerdata\_common.py#89\89]8;;\

In [3]:
%%capture  
#Scrape player stats for 2021/2022 and 2022/2023 season.
# player_match_stats_2021=fbref.read_player_match_stats(stat_type='summary')
player_match_stats=fbref1.read_player_match_stats(stat_type='summary')

ValueError: Length mismatch: Expected axis has 34 elements, new values have 36 elements

In [13]:
player_match_stats_2021.head(3)
player_match_stats.head(3)

#  \
                                                                                                        
league             season game                                  team           player                   
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton Crystal Palace Andros Townsend   10.0   
                                                                               Andros Townsend   10.0   
                                                                               Cheikhou Kouyaté   8.0   

                                                                                                  Nation  \
                                                                                                           
league             season game                                  team           player                      
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton Crystal Palace Andros Townsend   eng ENG   
                                                                               Andros Townsend   eng ENG   
                                                                               Cheikhou Kouyaté   sn SEN   

                                                                                                Pos  \
                                                                                                      
league             season game                                  team           player                 
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton Crystal Palace Andros Townsend   RM   
                                                                               Andros Townsend   RM   
                                                                               Cheikhou Kouyaté  CB   

                                                                                                    Age  \
                                                                                                          
league             season game                                  team           player                     
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton Crystal Palace Andros Townsend   29-058   
                                                                               Andros Townsend   29-058   
                                                                               Cheikhou Kouyaté  30-266   

                                                                                                  Min  \
                                                                                                        
league             season game                                  team           player                   
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton Crystal Palace Andros Townsend   90.0   
                                                                               Andros Townsend   90.0   
                                                                               Cheikhou Kouyaté  90.0   

                                                                                                Performance  \
                                                                                                        Gls   
league             season game                                  team           player                         
ENG-Premier League 2021   2020-09-12 Crystal Palace-Southampton Crystal Palace Andros Townsend          0.0   
                                                                               Andros Townsend          0.0   
                                                                               Cheikhou Kouyaté         0.0   

                                                                                                      \
                                                                                                 Ast   
league             season game                      

NameError: name 'player_match_stats' is not defined

Here we can see the data is presented in a multi-level index. This is highly difficult to work with so we'll be converting it to a single-level index. We will also reassign columns and merge our dataframes. 

In [ ]:
df_2021=player_match_stats_2021
df_2021=df_2021.droplevel(['league', 'season'])
df_2021.columns=df_2021.columns.droplevel(0)
df_2021.columns.values[0] = "#"
df_2021.columns.values[1] = 'Nation'
df_2021.columns.values[2] = 'Pos'
df_2021.columns.values[3] = 'Age'
df_2021.columns.values[4] = 'Mins'
df_2021.columns.values[28] = 'game_id'
df_2021=df_2021.reset_index()
#------------------------------------------------------------------------------------------------------------------------------#
df_2022=player_match_stats
df_2022=df_2022.droplevel(['league', 'season'])
df_2022.columns=df_2022.columns.droplevel(0)
df_2022.columns.values[0] = "#"
df_2022.columns.values[1] = 'Nation'
df_2022.columns.values[2] = 'Pos'
df_2022.columns.values[3] = 'Age'
df_2022.columns.values[4] = 'Mins'
df_2022.columns.values[28] = 'game_id'
df_2022=df_2022.reset_index()
#------------------------------------------------------------------------------------------------------------------------------#
dfFinal=pd.concat([df_2021, df_2022], ignore_index=True)
dfFinal.head()

Upon inspection, there is no column indicating a home/away fixture or the opposition team. However, when looking at the game column, the home team appears to always be the team listed first after the date, and the away team the second. We'll be leveraging this to make our own Location column to define home/away fixtures and make a new column called 'opposition'. Also, our game column will serve as a date column. 

In [ ]:
#Replicate game column to be used as a date column. Also removing the date from Location column.
dfFinal['Location'] = dfFinal['game']
dfFinal['Location'] = dfFinal['Location'].str[11:]  

In [ ]:
sep = '-'
def f1(x):
    return re.sub('[^0-9]','', x) #Replaces all characters that aren't numbers or hyphens to blank spaces.
def f2(x):
    return x.split(sep, 1)[1] #Deletes any character before a hyphen.
def f3(x):
    return x.split(sep, 1)[0] #Deletes any character after a hyphen.    

Apply f1 to 'game' column to become our 'date column' and sort our dataframe by date. 

Apply f2 to our location column to leave the second listed team 

Apply f3 to our location column to leave the first listed team 

In [ ]:
#Fixing up new date column.
dfFinal['game'] = dfFinal['game'].apply(f1)
dfFinal['game'] = pd.to_datetime(dfFinal['game'])
dfFinal=dfFinal.rename(columns={'game':'date'})
dfFinal=dfFinal.reset_index(drop=True)
dfFinal=dfFinal.sort_values(by='date', ascending=False)
#------------------------------------------------------------------------------------------------------------------------------#
#Creating column for second listed team.
dfFinal['team2'] = dfFinal['Location'].apply(f2)
#------------------------------------------------------------------------------------------------------------------------------#
#Fixing up location column based on first listed team.
dfFinal['Location'] = dfFinal['Location'].apply(f3)

dfFinal.head()

In order to reassign values in the location column to Home/Away, the names in the team and location column must have the exact same spelling. 

In [ ]:
dfFinal['team'].unique()
dfFinal['Location'].unique()

As we can see, some teams are named differently (e.g. Newcastle United vs Newcastle Utd).  This is resolved by manually renaming team spellings in the team column to those observed in the Location column's.

In [ ]:
df_2022['team'] = df_2022['team'].replace({'Wolverhampton Wanderers':'Wolves', 'Newcastle United':'Newcastle Utd', 'Tottenham Hotspur':'Tottenham', 'Brighton & Hove Albion':'Brighton', 'Manchester United':'Manchester Utd', 'West Ham United':'West Ham', 'Nottingham Forest':"Nott'ham Forest"})
dfFinal['team'] = dfFinal['team'].replace({'Wolverhampton Wanderers':'Wolves', 'Newcastle United':'Newcastle Utd', 'Tottenham Hotspur':'Tottenham', 'Brighton & Hove Albion':'Brighton', 'Manchester United':'Manchester Utd', 'West Ham United':'West Ham', 'Nottingham Forest':"Nott'ham Forest"})

In [ ]:
dfFinal['team'].unique()
dfFinal['Location'].unique()

We can now focus on creating the opposition column.

In [ ]:
#Create opposition column and give TRUE/FALSE values based on if Location/Team columns are the same.
dfFinal['Opposition'] = dfFinal['Location']
dfFinal = dfFinal.assign(Opposition = lambda x: (x['team']==x['Location']))
dfFinal.reset_index(drop = True, inplace= True)

#If TRUE, we know the opposition is the team listed in team2 column, otherwise it must be the team listed in location column. 
rowcount = 0 
for x in dfFinal['Opposition']:
    if x == True:
        dfFinal.at[rowcount, 'Opposition']=dfFinal.at[rowcount, 'team2']
    else:
        dfFinal.at[rowcount, 'Opposition']=dfFinal.at[rowcount, 'Location']
    rowcount = rowcount + 1

dfFinal.drop(columns=['team2'], inplace=True)

dfFinal.head()

That's better. Let's now convert the team name in location to either home/away. 

In [ ]:
#Test if value in column equals value in location/team (if TRUE - Home game, and if FALSE - Away game).
dfFinal = dfFinal.assign(Location = lambda x: (x['team']==x['Location']))

#Convert TRUE/FALSE boolean to string to allow for string replacements. 
mask = dfFinal.applymap(type) != bool
d = {True: 'TRUE', False: 'FALSE'}                                        
dfFinal = dfFinal.where(mask, dfFinal.replace(d))

#Replace TRUE/FALSE with Home/Away string.
dfFinal['Location'] = dfFinal['Location'].replace({'TRUE':'Home', 'FALSE':'Away'})

dfFinal.head()

Voila! Let's print out our final 3 dataframes. 

In [ ]:
dfPlayers.head(2)
dfFixtures.head(2)
dfFinal.head(2)

That's all of our dataframes cleaned up!

# 3. Data Engineering (Team Data)  

### npxG/npxGA Calculation
We will be applying a linear weighted mean over each team's npxG/npxGA over their 10 most recent games to predict future npxG/npxGA. The weighting chosen has been derived from previous testing. 

## 3.1. Team npxG

In [ ]:
#Collecting the npxG for each team in the current season for the last 10 games.
Teams=df_2022['team'].unique().tolist()
team_list=[]
npxG_Team_Weighted_Mean=[]

#Iterate through each team's previous 10 games via their game_id and store them as a list. 
for team in Teams: 
    dfTeam=dfFinal.loc[dfFinal['team'] == team]
    gamelist=(dfTeam['game_id'].unique().tolist())[:10] 
    gamelist=gamelist[::-1] 
    
#Go through each game_id and sum each player's npxG for said game and then multiply by a constant for our weighted average. 
    x=0 
    npxgsum=0
    sumweighting=0
    for game in gamelist:
        df2=dfTeam.loc[dfTeam['game_id'] == game]
        df2=df2.drop_duplicates().reset_index(drop=True)
        k=1+x
        x+=(1/9)
        npxgsum+=df2['npxG'].sum()*k 
        sumweighting=sumweighting+k 
    #Store results.
    npxGMean=npxgsum/sumweighting
    npxG_Team_Weighted_Mean.append(npxGMean)
    team_list.append(team)

Split our df into fixtures only containing home/away games.

In [ ]:
dfHome=dfFinal.loc[dfFinal['Location'] == 'Home']
dfAway=dfFinal.loc[dfFinal['Location'] == 'Away']

## 3.2. Team npxG (Home Fixtures)

In [ ]:
team_list1=[]
Home_npxG_Team_Weighted_Mean=[]

for team in Teams: 
    dfTeam=dfHome.loc[dfHome['team'] == team] #This is where we specify home games. All else is same as above. 
    gamelist=(dfTeam['game_id'].unique().tolist())[:10] 
    gamelist=gamelist[::-1] 
    
    Home_npxgsum=0
    x=0
    sumweighting=0
    for game in gamelist:
        df2=dfTeam.loc[dfTeam['game_id'] == game]
        df2=df2.drop_duplicates().reset_index(drop=True)
        k=1+x
        x+=(1/9)
        Home_npxgsum+=df2['npxG'].sum()*k
        sumweighting=sumweighting+k 
    
    Home_npxGMean=Home_npxgsum/sumweighting
    Home_npxG_Team_Weighted_Mean.append(Home_npxGMean)
    team_list1.append(team)

## 3.3. Team npxG (Away Fixtures)

In [ ]:
team_list2=[]
Away_npxG_Team_Weighted_Mean=[]

for team in Teams: 
    dfTeam=dfAway.loc[dfAway['team'] == team] #This is where we specify away games. All else is same as above.
    gamelist=(dfTeam['game_id'].unique().tolist())[:10] 
    gamelist=gamelist[::-1] 
    
    Away_npxgsum=0
    x=0
    sumweighting=0
    for game in gamelist:
        df2=dfTeam.loc[dfTeam['game_id'] == game]
        df2=df2.drop_duplicates().reset_index(drop=True)
        k=1+x
        x+=(1/9)
        Away_npxgsum+=df2['npxG'].sum()*k
        sumweighting=sumweighting+k 

    Away_npxGMean=Away_npxgsum/sumweighting
    Away_npxG_Team_Weighted_Mean.append(Away_npxGMean)
    team_list2.append(team)

## 3.4. Team npxGA 

Same process as above but summing npxG of the opposition to collect defensive stats (npxGA) for each team.

In [ ]:
team_list1=[]
npxGA_Team_Weighted_Mean=[]

for team in Teams: 
    dfTeam=dfFinal.loc[dfFinal['team'] == team]
    gamelist=(dfTeam['game_id'].unique().tolist())[:10] 
    gamelist=gamelist[::-1] 
     
    x=0 
    npxgasum=0
    sumweighting=0
    for game in gamelist:
        df2=dfFinal.loc[dfFinal['game_id'] == game]
        df2=df2.loc[df2['team'] != team] #Here we specify collecting npxG for opposition - in turn giving us the npxGA.
        df2=df2.drop_duplicates().reset_index(drop=True)
        k=1+x
        x+=(1/9)
        npxgasum+=df2['npxG'].sum()*k
        sumweighting=sumweighting+k 

    npxGAMean=npxgasum/sumweighting
    npxGA_Team_Weighted_Mean.append(npxGAMean)
    team_list1.append(team)

## 3.5. Team npxGA (Home Fixtures)

In [ ]:
team_list1=[]
Home_npxGA_Team_Weighted_Mean=[]

for team in Teams: 
    dfTeam=dfHome.loc[dfHome['team'] == team]
    gamelist=(dfTeam['game_id'].unique().tolist())[:10] 
    gamelist=gamelist[::-1] 
    
    Home_npxgasum=0
    x=0
    sumweighting=0
    for game in gamelist:
        df2=dfAway.loc[dfAway['game_id'] == game]
        df2=df2.loc[df2['team'] != team]
        df2=df2.drop_duplicates().reset_index(drop=True)
        k=1+x
        x+=(1/9)
        Home_npxgasum+=df2['npxG'].sum()*k
        sumweighting=sumweighting+k 
        
    Home_npxGAMean=Home_npxgasum/sumweighting
    Home_npxGA_Team_Weighted_Mean.append(Home_npxGAMean)
    team_list1.append(team)

## 3.6. Team npxGA (Away Fixtures)

In [ ]:
team_list2=[]
Away_npxGA_Team_Weighted_Mean=[]

for team in Teams: 
    dfTeam=dfAway.loc[dfAway['team'] == team]
    gamelist=(dfTeam['game_id'].unique().tolist())[:10] 
    gamelist=gamelist[::-1] 
    
    Away_npxgasum=0
    x=0
    sumweighting=0
    for game in gamelist:
        df2=dfHome.loc[dfHome['game_id'] == game]
        df2=df2.loc[df2['team'] != team]
        df2=df2.drop_duplicates().reset_index(drop=True)
        k=1+x
        x+=(1/9)
        Away_npxgasum+=df2['npxG'].sum()*k
        sumweighting=sumweighting+k   

    Away_npxGAMean=Away_npxgasum/sumweighting
    Away_npxGA_Team_Weighted_Mean.append(Away_npxGAMean)
    team_list1.append(team)

Let's combine our npxG and npxGA home/away data into a new dataframe for inspection. 

In [ ]:
df_att_team = pd.DataFrame((zip(team_list, npxG_Team_Weighted_Mean, Home_npxG_Team_Weighted_Mean, Away_npxG_Team_Weighted_Mean)),
               columns =['Team', 'npxG', 'Home npxG', 'Away npxG'])
df_def_team = pd.DataFrame((zip(team_list1, npxGA_Team_Weighted_Mean, Home_npxGA_Team_Weighted_Mean, Away_npxGA_Team_Weighted_Mean)),
               columns =['Team', 'npxGA', 'Home npxGA', 'Away npxGA'])
df_team=(pd.merge(df_att_team, df_def_team, on='Team'))
df_team.head()

Everything looks good! Note how home stats are typically better than away; higher npxG and lower npxGA i.e. they score more and concede less.

Next we will create Home/Away modifiers. 

# 4. Home/Away Modifiers for npxG/npxGA 

We will consider the impact oh Home/Away games for each team by creating modifiers for both npxG and npxGA


## 4.1. Home/Away Modifier Calculation

Our modifier will consist of two methods given equal weighting; 

1: A team's home/away performance relative to their base performance. 

2: A team's home/away performance relative to  other team's home/away performances (with a MinMax scaling applied to reduce the impact of more extreme results).

In [ ]:
#Modifier 1: A team's home/away performance relative to their base performance.
scaler = MinMaxScaler(feature_range=(0.95, 1.05)) #bring in a scaler to lessen effect of outliers. 
df_team['Att_Home_Modifier_1']=df_team['Home npxG']/df_team['npxG']
df_team['Att_Home_Modifier_1']=scaler.fit_transform(df_team['Att_Home_Modifier_1'].values.reshape(-1, 1))

df_team['Att_Away_Modifier_1']=df_team['Away npxG']/df_team['npxG']
df_team['Att_Away_Modifier_1']=scaler.fit_transform(df_team['Att_Away_Modifier_1'].values.reshape(-1, 1))

df_team['Def_Home_Modifier_1']=df_team['Home npxGA']/df_team['npxGA']
df_team['Def_Home_Modifier_1']=scaler.fit_transform(df_team['Def_Home_Modifier_1'].values.reshape(-1, 1))

df_team['Def_Away_Modifier_1']=df_team['Away npxGA']/df_team['npxGA']
df_team['Def_Away_Modifier_1']=scaler.fit_transform(df_team['Def_Away_Modifier_1'].values.reshape(-1, 1))

#Modifier 2: A team's home/away performance relative to  other team's home/away performances.
scaler = MinMaxScaler(feature_range=(0.9, 1.1)) #bring in a scaler to lessen effect of outliers. 
df_team['Att_Home_Modifier_2']=df_team['Home npxG']/(df_team['Home npxG'].median())
df_team['Att_Home_Modifier_2']=scaler.fit_transform(df_team['Att_Home_Modifier_2'].values.reshape(-1, 1))

df_team['Att_Away_Modifier_2']=df_team['Away npxG']/(df_team['Away npxG'].median())
df_team['Att_Away_Modifier_2']=scaler.fit_transform(df_team['Att_Away_Modifier_2'].values.reshape(-1, 1))

df_team['Def_Home_Modifier_2']=df_team['Home npxGA']/(df_team['Home npxGA'].median())     
df_team['Def_Home_Modifier_2']=scaler.fit_transform(df_team['Def_Home_Modifier_2'].values.reshape(-1, 1))

df_team['Def_Away_Modifier_2']=df_team['Away npxGA']/(df_team['Away npxGA'].median())
df_team['Def_Away_Modifier_2']=scaler.fit_transform(df_team['Def_Away_Modifier_2'].values.reshape(-1, 1))

#Final Modifier: Modifier 1 and 2 with equal weighting.                                                        
df_team['npxG_Final_Home_Multiplier']=(df_team['Att_Home_Modifier_1']+df_team['Att_Home_Modifier_2'])/2
df_team['npxG_Final_Away_Multiplier']=(df_team['Att_Away_Modifier_1']+df_team['Att_Away_Modifier_2'])/2
df_team['npxGA_Final_Home_Multiplier']=(df_team['Def_Home_Modifier_1']+df_team['Def_Home_Modifier_2'])/2
df_team['npxGA_Final_Away_Multiplier']=(df_team['Def_Away_Modifier_1']+df_team['Def_Away_Modifier_2'])/2

#Apply modifier to relevant columns.
df_team['Home_npxG']=df_team['Home npxG']*df_team['npxG_Final_Home_Multiplier']
df_team['Away_npxG']=df_team['Away npxG']*df_team['npxG_Final_Away_Multiplier']
df_team['Home_npxGA']=df_team['Home npxGA']*df_team['npxGA_Final_Home_Multiplier']
df_team['Away_npxGA']=df_team['Away npxGA']*df_team['npxGA_Final_Away_Multiplier']

#Clean up df.
df_Team_Final=df_team[['Team', 'npxG', 'Home_npxG', 'Away_npxG', 'npxGA', 'Home_npxGA', 'Away_npxGA', 'npxG_Final_Home_Multiplier', 'npxG_Final_Away_Multiplier', 'npxGA_Final_Home_Multiplier', 'npxGA_Final_Away_Multiplier']]
df_Team_Final.sort_values(by='npxGA', ascending=False, inplace=True)

Let's print out our new updated dataframe. 

In [ ]:
df_Team_Final.head()

## 4.2. Dictionaries

Various dictionaries for future use. 

In [ ]:
Dict_Att_Home_Mod=dict(zip(df_Team_Final.Team, df_Team_Final.npxG_Final_Home_Multiplier))
Dict_Att_Home=dict(zip(df_Team_Final.Team, df_Team_Final.Home_npxG)) 
Dict_Att_Away_Mod=dict(zip(df_Team_Final.Team, df_Team_Final.npxG_Final_Away_Multiplier))
Dict_Att_Away=dict(zip(df_Team_Final.Team, df_Team_Final.Away_npxG))
Dict_Def_Home=dict(zip(df_Team_Final.Team, df_Team_Final.npxGA_Final_Home_Multiplier))
Dict_Def_Away=dict(zip(df_Team_Final.Team, df_Team_Final.npxGA_Final_Away_Multiplier))

# 5. Fixture Tables

Next we'll move onto creating a fixture fixture table for each team for the next five GameWeeks. 

## 5.1. Home Fixture Table

In [ ]:
#Take the current gameweek and make a list of the upcoming five GameWeeks. 
Current_GameWeek=fixtures['GameWeek'].min()
Five_GameWeeks=np.arange(Current_GameWeek, Current_GameWeek+5, 1)

#Function to convert list into concatenated strings. 
def lts(s):
    str1 = ""
    for ele in s:
        str1 += ele
    return str1

awaylist=[]
awaylist1=[]
teamlist=[]

#Go through each team and find all their home fixtures for the next five GameWeeks.
for team in Teams:
    f1=fixtures[fixtures['Home Team'].str.contains(team)]
    awaylist=[]
    for i in Five_GameWeeks:
        f2=f1[f1['GameWeek']==i]
        
        if f2.shape[0]>0:
            away=f2.iloc[0,2] + ' ' + str(i) + ' - '
            awaylist.append(away)
    teamlist.append(team)
    awayfix=lts(awaylist)
    awaylist1.append(awayfix)

d = {'Team':teamlist,'Home Fixtures':awaylist1}
Home_Fixtures = pd.DataFrame(d)
Home_Fixtures.head()

We've captured the important data here. We just need to clean it up by seperating each gameweek into a column and placing the fixtures in the appropriate cell. 

In [ ]:
#Seperate each GameWeek into a column and place the fixtures for each team in their respective cells. 
def ics(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

dlist=[]
for g in Five_GameWeeks:
    dlist=[]
    for x in Home_Fixtures['Home Fixtures']:
        fixture = x.split(" - ")
        index = ics(fixture, str(g))
        d = fixture[index]
        d = re.sub(r'\d+', '', d)
        d = d.strip()
        dlist.append(d)
    Home_Fixtures['GW' + str(g) + ' Home'] = dlist
Home_Fixtures = Home_Fixtures.drop(columns=['Home Fixtures'])
Home_Fixtures.head()

## 5.2. Away Fixture Table

In [ ]:
homelist=[]
homelist1=[]
teamlist=[]

for team in Teams:
    f1=fixtures[fixtures['Away Team'].str.contains(team)]
    
    homelist=[]
    for i in Five_GameWeeks:
        f2=f1[f1['GameWeek']==i]
        if f2.shape[0]>0:
            home=f2.iloc[0,1] + ' ' + str(i) + ' - '
            homelist.append(home)
    teamlist.append(team)
    homefix=lts(homelist)
    homelist1.append(homefix)

d = {'Team':teamlist,'Away Fixtures':homelist1}
Away_Fixtures = pd.DataFrame(d)

#------------------------------------------------------------------------------------------------------------------------------#

dlist=[]
for g in Five_GameWeeks:
    dlist=[]
    for x in Away_Fixtures['Away Fixtures']:
        fixture = x.split(" - ")
        index = ics(fixture, str(g))
        d = fixture[index]
        d = re.sub(r'\d+', '', d)
        d = d.strip()
        dlist.append(d)
    Away_Fixtures['GW' + str(g) + ' Away'] = dlist
Away_Fixtures = Away_Fixtures.drop(columns=['Away Fixtures'])
Away_Fixtures.head()

## 5.3. Home Fixture Table - Insert Stats

Here we will be utilizing the dictionaries created earlier to insert attacking/defensive stats. 

In [ ]:
Home_Fixtures_Team=Home_Fixtures.copy()
Home_Fixtures_Team['Team npxG']=Home_Fixtures_Team['Team']

#Replace team with their respective defensive away stats. 
for g in Five_GameWeeks:
    Home_Fixtures_Team=Home_Fixtures_Team.replace({"GW" + str(g) + " Home": Dict_Def_Away})
    Home_Fixtures_Team=Home_Fixtures_Team.replace({"Team": Dict_Att_Home})
    
columncount=0
for g in Five_GameWeeks:
    rowcount=0
    columncount = columncount + 1    
    
    for x in Home_Fixtures_Team["GW" + str(g) + " Home"]:
        if x == "":
            Home_Fixtures_Team.iloc[rowcount, columncount]=0 #Replace blank spaces with 0. 
        else:
            Home_Fixtures_Team.iloc[rowcount, columncount]=Home_Fixtures_Team.iloc[rowcount, 0]*x #npxG modified by npxGA of away team
        rowcount = rowcount + 1
        
Home_Fixtures_Team.head()

## 5.4. Away Fixture Table - Insert Stats

In [ ]:
Away_Fixtures_Team=Away_Fixtures.copy()
Away_Fixtures_Team['Team npxG']=Away_Fixtures_Team['Team']
for g in Five_GameWeeks:
    Away_Fixtures_Team=Away_Fixtures_Team.replace({"GW" + str(g) + " Away": Dict_Def_Home})
    Away_Fixtures_Team=Away_Fixtures_Team.replace({"Team": Dict_Att_Away})

columncount=0
for g in Five_GameWeeks:
    rowcount=0
    columncount = columncount + 1    
    
    for x in Away_Fixtures_Team["GW" + str(g) + " Away"]:
        if x == "":
            Away_Fixtures_Team.iloc[rowcount, columncount]=0
        else:
            Away_Fixtures_Team.iloc[rowcount, columncount]=Away_Fixtures_Team.iloc[rowcount, 0]*x
        rowcount = rowcount + 1
Away_Fixtures_Team.head()

## 5.5. Finalizing Fixture Table

Let's merge Home and Away Fixtures for each team.

In [ ]:
Fixtures_Merged_Team=(pd.merge(Home_Fixtures_Team, Away_Fixtures_Team, how='left', on='Team npxG'))
Fixtures_Merged_Team.head()

Not very pretty, let's clean it up.  

In [ ]:
#Combine GW'x' Home and Away columns into 1 column
for g in Five_GameWeeks:
    Fixtures_Merged_Team['GW' + str(g)] = Fixtures_Merged_Team['GW' + str(g) + ' Home'] + Fixtures_Merged_Team['GW' + str(g) + ' Away']

#Send Team npxG to last column of dataframe
new_cols = [col for col in Fixtures_Merged_Team.columns if col != 'Team npxG'] + ['Team npxG']
Fixtures_Merged_Team = Fixtures_Merged_Team[new_cols]

#Remove all other columns apart from the relevant GW columns
Fixtures_Merged_Team = Fixtures_Merged_Team.iloc[:, -(np.count_nonzero(Five_GameWeeks)+1):]

#New npxG total columns based on the sum of all GW's and renaming to 'Team' column for clarity 
Fixtures_Merged_Team['npxG Total']=Fixtures_Merged_Team.iloc[:, 0:4].sum(axis=1)
Fixtures_Merged_Team=Fixtures_Merged_Team.rename(columns={'Team npxG':'Team'})

#Reposition team column to first column for clarity and sort by npxG total 
team = Fixtures_Merged_Team['Team']
Fixtures_Merged_Team.drop(labels=['Team'], axis=1,inplace = True)
Fixtures_Merged_Team.insert(0, 'Team', team)
Fixtures_Merged_Team.sort_values(by='npxG Total', ascending=False, inplace=True)
df_Team_Final_Fixture_Predictions = Fixtures_Merged_Team

df_Team_Final_Fixture_Predictions.head()

Much better - our final Teams-Prediction table! 

# 6. Data Engineering (Player Data) - Fuzzy Matching Included

We calculate the npxG for each player using the same steps used in the Data Engineering (Team Data) section and apply the same fixture tables. 

However, an extra step is required in our player data since the official FPL site and FBref have named certain players differently. This means when joining datasets between our two player dataframes from our different sources, a straight join between the two datasets will be infeasible. We will use fuzzy matching to resolve this. 

## 6.1. Player npxG and xAG

Very similar process to the one used in Team npxG. We will however include expected assisted goals (xAG) as well as npxG.


In [ ]:
Players=df_2022['player'].unique().tolist() #Every player in the current season.
player_list=[]
mins=[]
npxG_Player_Weighted_Mean=[]
xAG_Player_Weighted_Mean=[]

#Iterate to find each player's last 10 games.
for player in Players: 
    dfPlayer=dfFinal.loc[dfFinal['player']==player]
    dfPlayer=(dfPlayer.drop_duplicates().reset_index(drop=True))[:10]
    dfPlayer=dfPlayer.iloc[::-1]
    
    #Give weighting ranging from 1 to 2. 
    step=1/(dfPlayer.shape[0])
    weighting=(np.arange(1, 2, step)).tolist()
    dfPlayer['weighting']=weighting
    weightingsum=dfPlayer['weighting'].sum()
    totalmins=dfPlayer['Mins'].sum()
    
    npxG_Weighted=((dfPlayer['npxG']*dfPlayer['weighting']).sum())/weightingsum
    xAG_Weighted=((dfPlayer['xAG']*dfPlayer['weighting']).sum())/weightingsum
    
    #Store results
    mins.append(totalmins)
    npxG_Player_Weighted_Mean.append(npxG_Weighted)
    xAG_Player_Weighted_Mean.append(xAG_Weighted)
    player_list.append(player)

Make a dataframe from our fbref player data. 

In [ ]:
df_Players_fbref = pd.DataFrame((zip(player_list, npxG_Player_Weighted_Mean, xAG_Player_Weighted_Mean, mins)),
               columns =['Player', 'npxG/Appearance', 'xAG/Appearance', 'Total Minutes'])
df_Players_fbref['npxG + xAG/Appearance']=df_Players_fbref['npxG/Appearance']+df_Players_fbref['xAG/Appearance']
df_Players_fbref.sort_values(by='npxG + xAG/Appearance', ascending=False, inplace=True)
df_Players_fbref.head()

## 6.2. Fuzzy Matching

When merging our data with the player information from the official FPL site we encounter a problem. 

In [ ]:
df_player_final=(pd.merge(df_Players_fbref, dfPlayers_FPL, how='left', on='Player'))
df_player_final = df_player_final.reset_index()

df_player_final.head(5)

Alot of NaN's because not every player name perfectly matches between the two sets as proven below:

In [ ]:
dfPlayers_FPL_ManUtd=dfPlayers_FPL.loc[dfPlayers_FPL['Team'] == 'Man Utd']
print ('Official FPL: ' + str(dfPlayers_FPL_ManUtd.iloc[8][0]))
print ('FBref: ' + str(df_player_final.iloc[4][1]))

We will need to use fuzzy-matching to overcome this.

We will use two different algorithms. If the player match reaches a threshhold of 90% accuracy on either algorithm, we will presume the matching is correct. If not, we will manually cross-check with the suggestions our algorithms give us. 

In [ ]:
df_player_final_null = df_player_final[df_player_final['Team'].isna()] #Dataframe of players which returned null

First fuzzy-matching algorithm: 

In [ ]:
Suggested_Player = []
Score = []
PlayerList = dfPlayers['Player'].to_list()

#Reference each player from the official FPL site with the respective suggested player from FBref. 
for name_to_find in df_player_final_null['Player']:
    Suggested_P =  (process.extractOne(name_to_find, PlayerList))[0] #calling fuzzywords library to fuzzyword match.
    S = (process.extractOne(name_to_find, PlayerList))[1]
    Suggested_Player.append(Suggested_P)
    Score.append(S)

#Create dataframe from the lists created above. 
df_player_final_null['Suggested_Player']=Suggested_Player
df_player_final_null['Score']=Score
df_player_final_null=df_player_final_null.reset_index()
df_player_final_null=df_player_final_null[['Player', 'Suggested_Player', 'index', 'Score']]

df_player_final_null.head()

A score of 90 or above demonstrates very high similarity. 

Split matches between between above/below 90 score.

In [ ]:
df_player_final_null1=df_player_final_null.query('Score>=90') 
df_player_final_null2=df_player_final_null.query('Score<90')

Join players where suggested players >90 score.

In [ ]:
#Merging suggested players with >90 score.
df_player_final_null1=df_player_final_null1[['Suggested_Player', 'index']]
df_player_final1=(pd.merge(df_player_final, df_player_final_null1, how='left', on='index'))
df_player_final1.Suggested_Player.fillna(df_player_final1.Player, inplace=True)
df_player_final1=df_player_final1[['Suggested_Player', 'index', 'npxG/Appearance', 'xAG/Appearance', 'npxG + xAG/Appearance']]

df_player_final1.head()

Second fuzzy-matching algorithm:

In [ ]:
Suggested_Player = []
Score = []

for name_to_find in df_player_final_null2['Player']:
    Suggested_P =  (process.extractOne(name_to_find, PlayerList, scorer=fuzz.token_sort_ratio))[0]
    S = (process.extractOne(name_to_find, PlayerList, scorer=fuzz.token_sort_ratio))[1]
    Suggested_Player.append(Suggested_P)
    Score.append(S)
    
df_player_final_null2['Suggested_Player1']=Suggested_Player
df_player_final_null2['Score1']=Score
df_player_final_null2=df_player_final_null2.reset_index(drop=True)

df_player_final_null2.head()

Unfortunately at this point we have to manually review the remaining NaN. 

In [ ]:
df_player_final_null2.at[0, 'Player'] = 'Gabriel Fernando de Jesus'
df_player_final_null2.at[1, 'Player'] = 'Diogo Teixeira da Silva'
df_player_final_null2.at[2, 'Player'] = 'Diego Da Silva Costa'
df_player_final_null2.at[3, 'Player'] = 'Andreas Hoelgebaum Pereira'
df_player_final_null2.at[4, 'Player'] = 'Ilkay Gündogan'
df_player_final_null2.at[5, 'Player'] = 'Pedro Lomba Neto'
df_player_final_null2.at[6, 'Player'] = 'Lucas Torreira di Pascua'
df_player_final_null2.at[7, 'Player'] = 'Daniel Castelo Podence'
df_player_final_null2.at[8, 'Player'] = 'Emiliano Buendía Stati'
df_player_final_null2.at[9, 'Player'] = 'Fábio Ferreira Vieira'
df_player_final_null2.at[10, 'Player'] = 'Wilfried Gnonto'
df_player_final_null2.at[11, 'Player'] = 'João Filipe Iria Santos Moutinho'
df_player_final_null2.at[12, 'Player'] = 'Joe Ayodele-Aribo'
df_player_final_null2.at[13, 'Player'] = 'Tomas Soucek'
df_player_final_null2.at[14, 'Player'] = 'Bernardo Veiga de Carvalho e Silva'
df_player_final_null2.at[15, 'Player'] = 'Gonçalo Manuel Ganchinho Guedes'
df_player_final_null2.at[16, 'Player'] = 'Bobby De Cordova-Reid'
df_player_final_null2.at[17, 'Player'] = 'Carlos Henrique Casimiro'
df_player_final_null2.at[18, 'Player'] = 'Fábio Freitas Gouveia Carvalho'
df_player_final_null2.at[19, 'Player'] = 'Benjamin White'
df_player_final_null2.at[20, 'Player'] = 'Konstantinos Tsimikas'
df_player_final_null2.at[21, 'Player'] = 'Rasmus Kristensen'
df_player_final_null2.at[22, 'Player'] = 'Rúben da Silva Neves'
df_player_final_null2.at[23, 'Player'] = 'Mateo Kovacic'
df_player_final_null2.at[24, 'Player'] = 'Rúben Gato Alves Dias'
df_player_final_null2.at[25, 'Player'] = 'Thiago Emiliano da Silva'
df_player_final_null2.at[26, 'Player'] = 'Renan Augusto Lodi dos Santos'
df_player_final_null2.at[27, 'Player'] = 'Jonathan Castro Otto'
df_player_final_null2.at[28, 'Player'] = 'Cédric Alves Soares'
df_player_final_null2.at[29, 'Player'] = 'Toti António Gomes'
df_player_final_null2.at[30, 'Player'] = 'Joseph Gomez'
df_player_final_null2.at[31, 'Player'] = 'Rúben Nascimento Vinagre'
df_player_final_null2.at[32, 'Player'] = 'Fabio Henrique Tavares'
df_player_final_null2.at[33, 'Player'] = 'Nélson Cabral Semedo'
df_player_final_null2.at[34, 'Player'] = 'Lucas Rodrigues Moura da Silva'
df_player_final_null2.at[35, 'Player'] = 'Jorge Luiz Frello Filho'
df_player_final_null2.at[36, 'Player'] = 'Mateo Joseph Fernández'
df_player_final_null2.at[37, 'Player'] = 'José Malheiro de Sá'
df_player_final_null2.at[38, 'Player'] = 'Alex Mighten'
df_player_final_null2.at[39, 'Player'] = 'Marcus Oliveira Alencar'
df_player_final_null2.at[40, 'Player'] = 'Carlos Ribeiro Dias'
df_player_final_null2.at[41, 'Player'] = 'Lukasz Fabianski'
df_player_final_null2=df_player_final_null2[['Player', 'index']]

That should be all our players matched. Let's check. 

In [ ]:
#Merge Suggested Players with Players and cleaning up. 
df_player_final2=(pd.merge(df_player_final1, df_player_final_null2, how='left', on='index'))
df_player_final2.Player.fillna(df_player_final2.Suggested_Player, inplace=True)
df_player_final2=df_player_final2[['Player', 'npxG/Appearance', 'xAG/Appearance', 'npxG + xAG/Appearance']]
df_Player_Final=(pd.merge(df_player_final2, dfPlayers_FPL, how='left', on='Player'))
df_Player_Final['Team'] = df_Player_Final['Team'].replace({'Newcastle':'Newcastle Utd', 'Spurs':'Tottenham','Leeds':'Leeds United', 'Leicester':'Leicester City', 'Man City':'Manchester City', 'Man Utd':'Manchester Utd', "Nott'm Forest":"Nott'ham Forest"})

df_Player_Final.head()

The teams and positions for each player have successfully updated! 

# 7. Players Fixture Tables

Again, the same steps used in the Teams Fixture Tables will be used. We will be adjusting npxG and npxGA based on the home/away multipliers of the teams each player will face. 

## 7.1. Merge with Team Data

We merge our teams dataframe to update the npxG/xAG of each player using the home/away multipliers. 

In [ ]:
df_Player_Final=(pd.merge(df_Player_Final, df_Team_Final, how='left', on='Team'))
df_Player_Final.head()

In [ ]:
#Calculate our new npxG/xAG's for Home/Away Fixtures and do some cleaning. 

#npxG and xAG - Home
df_Player_Final['npxG/Appearance Home']=df_Player_Final['npxG/Appearance']*df_Player_Final['npxG_Final_Home_Multiplier']
df_Player_Final['xAG/Appearance Home']=df_Player_Final['xAG/Appearance']*df_Player_Final['npxG_Final_Home_Multiplier']

#npxG and xAG - Away
df_Player_Final['npxG/Appearance Away']=df_Player_Final['npxG/Appearance']*df_Player_Final['npxG_Final_Away_Multiplier']
df_Player_Final['xAG/Appearance Away']=df_Player_Final['xAG/Appearance']*df_Player_Final['npxG_Final_Away_Multiplier']

#npxG + xAG - Home = Away
df_Player_Final['npxG + xAG/Appearance Home']=df_Player_Final['npxG/Appearance Home']+df_Player_Final['xAG/Appearance Home']
df_Player_Final['npxG + xAG/Appearance Away']=df_Player_Final['npxG/Appearance Away']+df_Player_Final['xAG/Appearance Away']

#Df cleaning
df_Player_Final=df_Player_Final[['Player', 'Position', 'Team', 'npxG/Appearance', 'xAG/Appearance', 'npxG + xAG/Appearance', 'npxG/Appearance Home', 'xAG/Appearance Home', 'npxG + xAG/Appearance Home', 'npxG/Appearance Away', 'xAG/Appearance Away', 'npxG + xAG/Appearance Away']]
df_Player_Final.head()

Split our dataframe into Home/Away Games and collect the next 5 GW's for each Player. 

In [ ]:
df_Player_Final_Home=df_Player_Final[['Player', 'Position', 'Team', 'npxG/Appearance Home', 'xAG/Appearance Home', 'npxG + xAG/Appearance Home']]
df_Player_Final_Away=df_Player_Final[['Player', 'Position', 'Team', 'npxG/Appearance Away', 'xAG/Appearance Away', 'npxG + xAG/Appearance Away']]
df_Player_Final_Home=(pd.merge(df_Player_Final_Home, Home_Fixtures, how='left', on='Team'))
df_Player_Final_Away=(pd.merge(df_Player_Final_Away, Away_Fixtures, how='left', on='Team'))

df_Player_Final_Home=df_Player_Final_Home.dropna().reset_index(drop=True)
df_Player_Final_Away=df_Player_Final_Away.dropna().reset_index(drop=True)

df_Player_Final_Home.tail()

## 7.2. Home Fixture Tables

In [ ]:
#Home fixture npxG + xAG for each player for the next 5 GW's. 
for g in Five_GameWeeks:
    df_Player_Final_Home=df_Player_Final_Home.replace({"GW" + str(g) + " Home": Dict_Def_Away})   
    
#Replace empty strings with 0.
columncount=5
for g in Five_GameWeeks:
    rowcount=0
    columncount = columncount + 1    
    
    for x in df_Player_Final_Home["GW" + str(g) + " Home"]:
        if x == "":
            df_Player_Final_Home.iloc[rowcount, columncount]=np.float64(0) #specify a numpy 0 since these return "0" when we divide by 0.
        rowcount=rowcount + 1 

#Make new columns where dividing npxG and xAG by the npxGA multiplier 
columncount=6
for g in Five_GameWeeks:   
        npxG_Multiplied=df_Player_Final_Home.iloc[:,3]*df_Player_Final_Home.iloc[:,columncount]
        df_Player_Final_Home["GW" + str(g) + " Home npxG"]=npxG_Multiplied
        columncount=columncount+1


columncount=6
for g in Five_GameWeeks:
    xAG_Multiplied=df_Player_Final_Home.iloc[:,4]*df_Player_Final_Home.iloc[:,columncount]
    df_Player_Final_Home["GW" + str(g) + " Home xAG"]=xAG_Multiplied
    columncount+=1

# #Data cleaning and making sum columns 
df_Player_Final_Home = df_Player_Final_Home.drop(df_Player_Final_Home.columns[2:11], axis=1)
df_Player_Final_Home.head()

## 7.3. Away Fixture Tables

In [ ]:
#Away fixture npxG + xAG for each player for the next 5 GW's. 
for g in Five_GameWeeks:
    df_Player_Final_Away=df_Player_Final_Away.replace({"GW" + str(g) + " Away": Dict_Def_Home})   
    
#replace empty strings with 0.
columncount=5
for g in Five_GameWeeks:
    rowcount=0
    columncount = columncount + 1    
    
    for x in df_Player_Final_Away["GW" + str(g) + " Away"]:
        if x == "":
            df_Player_Final_Away.iloc[rowcount, columncount]=np.float64(0)
        rowcount=rowcount + 1 
        
#Make new columns where multiplying npxG and xAG by the npxGA multiplier 
columncount=6
for g in Five_GameWeeks:   
    npxG_Multiplied=df_Player_Final_Away.iloc[:,3]*df_Player_Final_Away.iloc[:,columncount]
    df_Player_Final_Away["GW" + str(g) + " Away npxG"]=npxG_Multiplied
    columncount=columncount+1

columncount=6
for g in Five_GameWeeks:
    xAG_Multiplied=df_Player_Final_Away.iloc[:,4]*df_Player_Final_Away.iloc[:,columncount]
    df_Player_Final_Away["GW" + str(g) + " Away xAG"]=xAG_Multiplied
    columncount+=1
 
#data cleaning and making sum columns 
df_Player_Final_Away = df_Player_Final_Away.drop(df_Player_Final_Away.columns[2:11], axis=1)
df_Player_Final_Away.head()

## 7.4. Finalizing Fixture Table

Replace inf values with 0. 

In [ ]:
df_Player_Final_Home_Zeroes = df_Player_Final_Home.replace([np.inf], 0)  
df_Player_Final_Away_Zeroes = df_Player_Final_Away.replace([np.inf], 0)  

Merge Home and Away Fixtures.  

In [ ]:
df_Player_Final_Fixtures=(pd.merge(df_Player_Final_Home_Zeroes, df_Player_Final_Away_Zeroes, how='left', on='Player'))
df_Player_Final_Fixtures.head()

Combine GW 'x' Home and Away columns into 1 column. 

In [ ]:
for g in Five_GameWeeks:
    df_Player_Final_Fixtures['GW' + str(g) + ' npxG'] = df_Player_Final_Fixtures['GW' + str(g) + ' Home npxG'] + df_Player_Final_Fixtures['GW' + str(g) + ' Away npxG']
    df_Player_Final_Fixtures['GW' + str(g) + ' xAG'] = df_Player_Final_Fixtures['GW' + str(g) + ' Home xAG'] + df_Player_Final_Fixtures['GW' + str(g) + ' Away xAG']
df_Player_Final_Fixtures.head()

General cleanup for clarity. 

In [ ]:
#Deleting unneeded columns and new columns for Total npxG, xAG and npxG + xAG. Also reordering column list

#Create lists for reordering column purposes
Player_list=df_Player_Final_Fixtures['Player'] 
Position_list=df_Player_Final_Fixtures['Position_x']

#Specify last 10 columns
df_Player_Final_Fixtures=df_Player_Final_Fixtures.iloc[:,-10:]

#Sum metrics
df_Player_Final_Fixtures['Total npxG']=df_Player_Final_Fixtures.iloc[:, 0:5].sum(axis=1)
df_Player_Final_Fixtures['Total xAG']=df_Player_Final_Fixtures.iloc[:, 5:10].sum(axis=1)
df_Player_Final_Fixtures['Total npxG + xAG']=df_Player_Final_Fixtures['Total npxG']+df_Player_Final_Fixtures['Total xAG']

#Sorting
df_Player_Final_Fixtures.sort_values(by='Total npxG + xAG', ascending=False, inplace=True)

#Reorder columns
df_Player_Final_Fixtures.insert(0, 'Player', Player_list)
df_Player_Final_Fixtures.insert(0, 'Position', Position_list)

#Delete duplicates
df_Player_Final_Fixtures=df_Player_Final_Fixtures.drop_duplicates()

#Impute NaN with 0's. 
df_Player_Final_Fixtures=df_Player_Final_Fixtures.fillna(0)

Whew! Let's check out our final dataframe for predicting player npxG/XAG over the next five gameweeks.

In [ ]:
df_Player_Final_Fixtures.head()

# 8. Final Dataframes 

## 8.1. Team Stats

### Ordered by npxG

In [ ]:
df_Team_Final.sort_values(by='npxG', ascending=False)

### Ordered by npxGA

In [ ]:
df_Team_Final.sort_values(by='npxGA', ascending=True)

## 8.2. Team Fixture Tables

### Home Fixtures

In [ ]:
Home_Fixtures

### Away Fixtures

In [ ]:
Away_Fixtures

## 8.3. Team npxG Predictions 

In [ ]:
df_Team_Final_Fixture_Predictions

## 8.4. Player npxG and xAG Predictions

### All Players 

In [ ]:
df_Player_Final_Fixtures.head(15)

### Forward Players

In [ ]:
df_Player_Final_Fixtures_FWD=df_Player_Final_Fixtures.loc[df_Player_Final_Fixtures['Position']=='FWD']
df_Player_Final_Fixtures_FWD.head(15)

### Midfield Players 

In [ ]:
df_Player_Final_Fixtures_MID=df_Player_Final_Fixtures.loc[df_Player_Final_Fixtures['Position']=='MID']
df_Player_Final_Fixtures_MID.head(15)

### Defender Players 

In [ ]:
df_Player_Final_Fixtures_DEF=df_Player_Final_Fixtures.loc[df_Player_Final_Fixtures['Position']=='DEF']
df_Player_Final_Fixtures_DEF.head(15)